# How MongoEngine works for existing data
docs: http://docs.mongoengine.org/index.html

data source: https://github.com/pyKap-data/pyKap-247sports-cfb/tree/main/247sports-2014-player-recruit-data

#### First, I would have insert the raw data into the database

In [2]:
"""

Insert the data collected from 247 Sports into MongoDB
Author: Manasi Godse

"""

from pymongo import MongoClient
import json
import os

# Making Connection
client = MongoClient("mongodb://localhost:27017/")

# database
db = client["ckids"]

Collection = db["players247"]

data_dir="C:/Users/heyin/2020py/USC/Football/Football-Recruits-125Items"
data_files=[data_dir+"/"+i for i in os.listdir(data_dir) if ".json" in i]

for players247 in data_files:
    # Loading or Opening the json file
    with open(players247, 'rb') as file:
        try:
            file_data = json.load(file)
        except UnicodeDecodeError:
            continue

    # Inserting the loaded data in the Collection
    if isinstance(file_data, list):
        Collection.insert_many(file_data)

#### Then define models to model the data collected from 247 Sports

In [3]:
"""

Author: Manasi Godse

How was the model defined? -> The fields that are required currently are put into the model.
Pre-requisite: Data is already present in the MongoDB
1. Define a Dynamic Document which tells mongoengine that is an existing document.
2. Define the model as per the structure of the document by using only the required fields.
3. There is no need to write all the attributes from the model.
4. Define Embedded Documents for every nested document in the main document.
5. When adding a new attribute in the model, do one of the following:
    a. add the attribute in the main document class if its an attribute which is not nested in the original document.
    b. add the attribute in EmbeddedDocument class if its an attribute which belongs to a already modelled embedded class.
    c. add a new class for EmbeddedDocument if the attribute to be added is nested but does not belong to existing embedded classes.

"""

from mongoengine import *

# Connect to the DB
connect(host="mongodb://127.0.0.1:27017/ckids")


class Hometown(EmbeddedDocument):
    State = StringField()
    City = StringField()


# Only one attribute present.
class PlayerHighSchool(EmbeddedDocument):
    Name = StringField(blank=True)


# Only one attribute present.
class PrimaryPlayerPosition(EmbeddedDocument):
    Abbreviation = StringField(blank=True)


# Not all attributes of nested 'Player' document is used
# Hence, define it as a Dynamic Embedded Document.
class Player(DynamicEmbeddedDocument):
    FirstName = StringField(required=True)
    LastName = StringField(required=True)
    FullName = StringField()
    DefaultName = StringField()
    Height = StringField(blank=True)
    Weight = FloatField(blank=True)
    Birthdate = DateTimeField(blank=True)
    NationalRank = IntField(blank=True)
    StateRank = IntField(blank=True)
    TwitterContact = IntField(blank=True)
    Bio = StringField(blank=True)
    ScoutEvaluation = StringField(blank=True)
    Hometown = EmbeddedDocumentField(Hometown)
    PlayerHighSchool = EmbeddedDocumentField(PlayerHighSchool)
    PrimaryPlayerPosition = EmbeddedDocumentField(PrimaryPlayerPosition)


# Main class. Since the attributes we are going to use
# is part of nested Player document, define it as an attribute here
class players247(DynamicDocument):
    meta = {'collection': 'players247'}
    Player = EmbeddedDocumentField(Player)

In [4]:
records = players247.objects()
for record in records:
    # Example to print the States of all players
    print(record.Player.Hometown.State)
#     print(record.Player.Hometown)

Louisiana
Texas
New Jersey
Louisiana
Virginia
Virginia
California
Arizona
Texas
Louisiana
Alabama
Virginia
Florida
Alabama
District Of Columbia
Alabama
Louisiana
Georgia
California
Florida
California
Georgia
Alabama
Florida
Florida
Texas
Alabama
Maryland
Illinois
Texas
Michigan
Texas
Georgia
California
Louisiana
Michigan
Tennessee
California
Tennessee
South Carolina
Texas
Georgia
Florida
North Carolina
Florida
Florida
Mississippi
North Carolina
Texas
Louisiana
California
Ohio
Washington
Illinois
Ohio
New York
California
California
Illinois
Texas
New Jersey
Florida
Kansas
Tennessee
Ohio
Texas
Texas
Michigan
Florida
Florida
Georgia
Alabama
Iowa
Florida
Louisiana
Virginia
Florida
Iowa
Arkansas
New York
Arkansas
Oklahoma
Louisiana
Louisiana
California
Arizona
Texas
Louisiana
Texas
Texas
Missouri
Colorado
Texas
Florida
Mississippi
Indiana
Florida
Virginia
Texas
Pennsylvania
California
Utah
Georgia
Louisiana
North Carolina
California
Georgia
Arizona
Oklahoma
Tennessee
Florida
Texas
Virginia


Texas
New Jersey
North Carolina
Ohio
California
Arkansas
Florida
Texas
North Carolina
California
Georgia
Georgia
Pennsylvania
New Jersey
Massachusetts
Michigan
Kansas
Missouri
Texas
Alabama
California
Texas
Mississippi
North Carolina
Texas
District Of Columbia
Texas
California
Florida
Florida
Florida
Florida
Georgia
Georgia
Arizona
Florida
Illinois
California
Georgia
Florida
Oklahoma
Illinois
California
Florida
Florida
Georgia
Texas
Georgia
Missouri
Florida
Missouri
California
Michigan
Georgia
Mississippi
Florida
North Carolina
Mississippi
Florida
Texas
Texas
Florida
Florida
Ohio
Arkansas
Georgia
California
South Carolina
Georgia
California
Virginia
Connecticut
Texas
Pennsylvania
California
Texas
Louisiana
Florida
California
Arizona
Connecticut
California
South Carolina
Ohio
Michigan
Ohio
Texas
Ohio
Florida
California
Utah
Texas
California
Florida
Maryland
California
Louisiana
Delaware
South Carolina
Tennessee
Georgia
California
Florida
Michigan
Florida
Maryland
Connecticut
Pennsylvani

Illinois
Tennessee
California
Ohio
Texas
Arkansas
Ohio
Texas
Virginia
Georgia
Nevada
Georgia
Ohio
Florida
Texas
Florida
Pennsylvania
Alabama
Illinois
Texas
Texas
Texas
Texas
Texas
Texas
Illinois
Ohio
Texas
Texas
Florida
Mississippi
Florida
Colorado
Florida
California
Texas
Arkansas
Alabama
Indiana
Louisiana
Florida
Texas
Florida
Florida
Florida
Texas
Michigan
Nebraska
Florida
Ohio
Georgia
Michigan
Florida
Wisconsin
Texas
Georgia
New York
California
Louisiana
Mississippi
Michigan
Alabama
Florida
Florida
Texas
Texas
Louisiana
California
California
Louisiana
Texas
California
Alabama
Pennsylvania
California
Georgia
Texas
Florida
South Carolina
Alabama
Alabama
Mississippi
Pennsylvania
Mississippi
Virginia
Massachusetts
Virginia
Ohio
Texas
Hawaii
California
Michigan
Illinois
California
Arizona
New York
New Jersey
California
California
Georgia
Alabama
Ontario
Georgia
Georgia
Georgia
Texas
Texas
Florida
Texas
California
Texas
New Jersey
Michigan
Hawaii
Texas
Arizona
Arizona
Oregon
Iowa
Oklahom

Louisiana
Georgia
Florida
Colorado
Tennessee
Oklahoma
Texas
Georgia
Florida
Florida
Texas
Florida
North Carolina
Michigan
Florida
Florida
Ohio
Louisiana
Ohio
Arizona
California
Mississippi
Iowa
Florida
Texas
Texas
California
Texas
California
Texas
Florida
Texas
Texas
Hawaii
Texas
Texas
Georgia
Louisiana
Texas
Michigan
Wisconsin
Michigan
Texas
Texas
Texas
Washington
Texas
Iowa
Connecticut
California
Virginia
Florida
Pennsylvania
New York
Texas
Alabama
Virginia
Florida
New Mexico
South Carolina
Florida
Florida
Georgia
California
Texas
North Carolina
Louisiana
Missouri
Louisiana
Kentucky
Louisiana
Louisiana
Louisiana
Florida
Louisiana
Louisiana
Georgia
Tennessee
California
Michigan
Florida
Virginia
Maryland
Florida
Florida
Louisiana
California
Massachusetts
Massachusetts
Missouri
Kansas
North Dakota
Georgia
Texas
North Carolina
Georgia
Mississippi
Georgia
Georgia
Louisiana
Alabama
Mississippi
Florida
Texas
Florida
Illinois
Florida
Florida
Michigan
Texas
Louisiana
New York
California
Calif

New York
Texas
Arizona
California
Texas
Texas
Texas
Tennessee
Louisiana
North Carolina
Georgia
Georgia
Ohio
Ohio
Tennessee
Tennessee
Georgia
Georgia
Tennessee
Tennessee
Florida
Wisconsin
Louisiana
Florida
Georgia
Minnesota
Kansas
Texas
Texas
Georgia
Georgia
New York
Mississippi
Louisiana
New York
New York
Indiana
Ohio
Ohio
New Jersey
Florida
Texas
Louisiana
Texas
Florida
Georgia
Texas
Mississippi
Georgia
Texas
North Carolina
California
Georgia
Georgia
South Carolina
Virginia
Indiana
Virginia
North Carolina
Virginia
Texas
Indiana
Virginia
New Jersey
South Carolina
North Carolina
Maryland
Ohio
Virginia
Georgia
Alabama
Nevada
Florida
Virginia
New Jersey
Georgia
Ohio
Hawaii
Virginia
Virginia
Virginia
Massachusetts
Florida
Maryland
North Carolina
South Carolina
Virginia
Virginia
Virginia
North Carolina
South Carolina
Virginia
New York
Pennsylvania
Maryland
Mississippi
Virginia
Massachusetts
Florida
Arizona
Texas
Georgia
Tennessee
Idaho
Minnesota
Massachusetts
Maryland
Virginia
District Of C

...

Pennsylvania
Tennessee
Washington
Virginia
Alabama
New Jersey
Texas
Nevada
California
Mississippi
Nevada
California
Mississippi
California
California
California
Texas
Oklahoma
Texas
Michigan
Florida
Pennsylvania
Texas
Minnesota
Virginia
Florida
Wisconsin
Alabama
Georgia
Utah
Texas
Florida
Oklahoma
Hawaii
Washington
Washington
California
Michigan
Wisconsin
Washington
Illinois
Kansas
South Dakota
Wisconsin
Texas
Washington
North Carolina
North Carolina
Pennsylvania
Michigan
Texas
Pennsylvania
Washington
Mississippi
Texas
Pennsylvania
Pennsylvania
Missouri
Pennsylvania
Pennsylvania
California
California
Oregon
Michigan
Florida
New Jersey
Maryland
Texas
Mississippi
California
Texas
Texas
Texas
Texas
Illinois
Texas
California
Texas
Missouri
Ohio
Illinois
New Jersey
Ohio
Kansas
Michigan
Ohio
Illinois
North Carolina
Oklahoma
Florida
California
Pennsylvania
District Of Columbia
Texas
Texas
Texas
Pennsylvania
California
Oklahoma
Massachusetts
Texas
Arizona
Australia
Alabama
Mississippi
Nevada
M

Georgia
Missouri
Texas
California
New Jersey
Nebraska
Maryland
Virginia
Missouri
Oregon
Texas
Kansas
Minnesota
Texas
Nevada
Ohio
Florida
Tennessee
New York
Alabama
North Carolina
California
Missouri
Oklahoma
Oklahoma
Ohio
Michigan
Mississippi
Washington
Texas
Alabama
Georgia
Pennsylvania
Kentucky
Kentucky
Wisconsin
Florida
California
Washington
Mississippi
Louisiana
North Carolina
Florida
Georgia
Florida
Florida
Georgia
Massachusetts
Alabama
Iowa
Georgia
Arizona
Arkansas
Pennsylvania
Texas
California
Mississippi
Florida
Ohio
Hawaii
Louisiana
Louisiana
Texas
Louisiana
California
Texas
Hawaii
Texas
Texas
Louisiana
Texas
Louisiana
Texas
Texas
South Carolina
Virginia
Ontario
Texas
Florida
Texas
Texas
Georgia
Florida
Texas
Nevada
South Carolina
Louisiana
Ohio
Michigan
Ohio
California
New Jersey
New York
Arkansas
Florida
Connecticut
Missouri
Texas
North Carolina
Texas
North Carolina
Illinois
North Carolina
Georgia
Mississippi
Illinois
Minnesota
Pennsylvania
California
District Of Columbia
Io

Wisconsin
Texas
Texas
Texas
Texas
Kansas
Georgia
North Carolina
Tennessee
Mississippi
Florida
Maryland
Pennsylvania
Texas
California
Florida
California
Tennessee
Michigan
Mississippi
North Carolina
Texas
Michigan
Georgia
Michigan
Florida
Denmark
Texas
Georgia
Mississippi
New Jersey
New Jersey
Nevada
Arizona
Nebraska
Pennsylvania
New Jersey
New Jersey
Kansas
Ohio
Ohio
Michigan
Connecticut
New York
Arizona
Georgia
Wisconsin
New Jersey
Florida
Mississippi
Georgia
Oklahoma
Maryland
Maryland
Illinois
Florida
Florida
Oregon
Ohio
Ohio
Arizona
Louisiana
Arkansas
Nevada
Oklahoma
Georgia
Florida
Texas
Georgia
Florida
Texas
District Of Columbia
Maryland
Pennsylvania
Illinois
Illinois
Alabama
Iowa
California
Massachusetts
Connecticut
Texas
Indiana
Georgia
Georgia
Alabama
Mississippi
Kentucky
New Mexico
Texas
Florida
Tennessee
Texas
Washington
North Carolina
Arizona
Ohio
California
California
Maryland
West Virginia
New Mexico
New Mexico
Washington
Texas
California
Alabama
Idaho
Ohio
Florida
Colorad

California
California
Kentucky
North Carolina
California
Florida
Texas
Washington
Tennessee
Michigan
Alabama
California
California
South Carolina
Pennsylvania
Florida
Louisiana
Illinois
Texas
California
California
Florida
Washington
Georgia
Texas
South Carolina
Texas
Nebraska
Florida
Texas
Washington
Washington
Indiana
Pennsylvania
Florida
Texas
South Dakota
Washington
Oregon
Colorado
South Carolina
New Jersey
New Mexico
California
Colorado
Texas
Louisiana
Texas
Texas
Pennsylvania
Germany
Pennsylvania
Florida
Florida
Florida
Ohio
Texas
Florida
Idaho
California
Georgia
Florida
North Carolina
Tennessee
Florida
Oregon
Mississippi
Florida
Texas
North Carolina
Texas
Louisiana
Texas
Alabama
Kansas
Texas
California
South Carolina
Wisconsin
Alabama
Texas
Oregon
Ohio
South Carolina
Hawaii
Oregon
Pennsylvania
Texas
Kansas
North Carolina
North Carolina
California
Oregon
California
Connecticut
Oregon
California
Utah
Iowa
Oregon
Texas
California
California
Maryland
California
Missouri
Pennsylvania


Ohio
Hawaii
Washington
Mississippi
Georgia
Michigan
California
Georgia
Washington
Florida
Georgia
Ohio
Ohio
Florida
California
Texas
Minnesota
Washington
Texas
Florida
New Jersey
Texas
Indiana
Louisiana
Texas
Alaska
Pennsylvania
Georgia
Texas
Georgia
Indiana
California
Washington
California
Florida
Indiana
Texas
Pennsylvania
North Carolina
North Carolina
Washington
Arizona
Colorado
Indiana
Alabama
Washington
Georgia
California
Georgia
Florida
Georgia
California
Pennsylvania
Michigan
Louisiana
Connecticut
Iowa
Illinois
Massachusetts
Washington
Washington
Massachusetts
California
Pennsylvania
California
Maryland
California
California
Florida
Florida
Indiana
Texas
Florida
Maryland
Georgia
Florida
Florida
Arkansas
California
Texas
Tennessee
Florida
Utah
Texas
Pennsylvania
California
Alabama
Mississippi
California
Texas
Illinois
Michigan
California
Indiana
Pennsylvania
Mississippi
Missouri
California
Washington
Pennsylvania
Texas
Minnesota
Washington
Washington
California
Oregon
Texas
Calif

In [6]:
type(players247)

mongoengine.base.metaclasses.TopLevelDocumentMetaclass

## Filter data

###### Available operators are as follows:

ne – not equal to

lt – less than

lte – less than or equal to

gt – greater than

gte – greater than or equal to

not – negate a standard check, may be used before other operators (e.g. Q(age__not__mod=(5, 0)))

in – value is in list (a list of values should be provided)

nin – value is not in list (a list of values should be provided)

mod – value % x == y, where x and y are two provided values

all – every item in list of values provided is in array

size – the size of the array is

exists – value for field exists





###### The following operators are available as shortcuts to querying with regular expressions:

exact – string field exactly matches value

iexact – string field exactly matches value (case insensitive)

contains – string field contains value

icontains – string field contains value (case insensitive)

startswith – string field starts with value

istartswith – string field starts with value (case insensitive)

endswith – string field ends with value

iendswith – string field ends with value (case insensitive)

wholeword – string field contains whole word

iwholeword – string field contains whole word (case insensitive)

regex – string field match by regex

iregex – string field match by regex (case insensitive)

match – performs an $elemMatch so you can match an entire document within an array

In [7]:
# Example filter
result = players247.objects.filter(Player__Hometown__State="Florida")
print(result.count())

result_height = players247.objects.filter(Player__Height__gt=0)
print(result_height.count())

result_first = players247.objects.only('Player.FirstName')
print(result_first.count())

result_last = players247.objects.only('Player.LastName')
print(result_last.count())
# we can also use .sum(),.average()...

result_full = players247.objects.only('Player.FullName')
full_list = []
height_list = []

for full in result_full:
    full_list.append(full.Player.FullName)
    
for height in result_height:
    height_list.append(height.Player.Height)
    
print(full_list)

6737
0
60816
60816
['Leonard Fournette', 'Myles Garrett', 'Jabrill Peppers', 'Cam Robinson', "Da'Shawn Hand", 'Quin Blanding', "Adoree' Jackson", 'Kyle Allen', 'Tony Brown', 'Speedy Noil', 'Marlon Humphrey', 'Andrew Brown', 'Dalvin Cook', 'Rashaan Evans', 'Teez Tabor', 'Bo Scarbrough', 'Malachi Dupre', 'Lorenzo Carter', 'Joe Mixon', 'Sony Michel', 'JuJu Smith-Schuster', 'Raekwon McMillan', 'Roc Thomas', 'Ermon Lane', 'Chad Thomas', 'Solomon Thomas', "Tre' Williams", 'Damian Prince', 'Clifton Garrett', 'KD Cannon', 'Malik McDowell', 'Jamal Adams', 'Nick Chubb', 'Royce Freeman', 'Gerald Willis', 'Damon Webb', 'Josh Malone', 'Damien Mama', 'Jalen Hurd', 'Christian Miller', 'Ed Paris', 'Deshaun Watson', 'Travis Rudolph', 'Elijah Hood', 'Joseph Yearby', 'Kc McDermott', 'Rod Taylor', 'Will Grier', 'Davion Hall', 'Hootie Jones', 'Keller Chryst', 'Dante Booker', 'Budda Baker', 'Nyles Morgan', 'Marshon Lattimore', 'Curtis Samuel', 'Michiah Quick', 'Jeff Jones', 'Jamarco Jones', 'Nick Harvey', '

In [8]:
players247.objects.item_frequencies('Player.Hometown.State')

{'Victoria': 9,
 'Netherlands': 4,
 'Ohio': 2271,
 'Quebec': 26,
 'United Kingdom': 1,
 'California': 6898,
 'Tennessee': 1527,
 'South Carolina': 1185,
 'Missouri': 735,
 'Utah': 538,
 'England': 7,
 'Maine': 17,
 'Baden-WÃ¼rttemberg': 2,
 'Mississippi': 1666,
 'New South Wales': 8,
 'Kansas': 759,
 'Michigan': 1740,
 'Colorado': 437,
 'Oregon': 462,
 'West Virginia': 174,
 'Wyoming': 34,
 'North Dakota': 51,
 'Indiana': 1068,
 'Illinois': 1469,
 'Louisiana': 1934,
 'Kentucky': 551,
 None: 24,
 'Guam': 1,
 'Serbia': 1,
 'Mexico': 1,
 'France': 2,
 'District Of Columbia': 203,
 'Massachusetts': 373,
 'South Dakota': 31,
 'Maryland': 1016,
 'Vermont': 2,
 'Canada': 23,
 'Rhode Island': 20,
 'Denmark': 2,
 'Na': 16,
 'Nova Scotia': 1,
 'Nebraska': 324,
 'Queensland': 2,
 'Texas': 7973,
 'Nevada': 315,
 'Wisconsin': 517,
 'Idaho': 193,
 'Montana': 40,
 'New Hampshire': 32,
 'Puerto Rico': 1,
 'Manitoba': 2,
 'Hawaii': 314,
 'China, Peoples Republic of': 1,
 'North Carolina': 1725,
 'Delaw

In [9]:
result_first_a = players247.objects.filter(Player__FirstName__startswith='D')
for each in result_first_a:
    print(each.Player.FirstName)

Da'Shawn
Dalvin
Damian
Damon
Damien
Deshaun
Davion
Dante
Derick
Drake
David
D'haquille
Dylan
Dillon
Derrick
Demetrius
Dalton
Dante
Dewayne
Drew
David
Derrell
Dominick
Demarcus
Dravon
Davon
DeShawn
Donell
Deondre
Dontavius
DJ
Demarre
Dexter
Derwin
Delvin
Duke
Duke
D.J.
Daniel
Derek
Devin
Dallis
DaVonte
Derek
Devon
DeAndre
Donte
Darius
Dontavius
Dominique
Dyshon
DeShone
Denzil
Derrick
Darrel
D'Andre
Dareian
D.J.
Drue
Devante
Daniel
D.J.
Dwight
Darrion
D'Vonta
Demario
Dalvon
Demetrius
D.J.
David
DeMarquis
Donnie
D'Ernest
Deon
Dylan
Dayall
Darrion
Detric
Dallas
Deshaun
Darrell
Darin
Derek
D.K.
Davonte
Dalton
Datrin
DaeJuan
Daniel
Daniel
Darrius
DeAndre
De'Eric
David
Dorian
Don
Derrick
Dimarya
Deion
De'Chavon
Dalvin
Devin
Darryl
Denzel
Dontae
Demarlon 
Darrion
Daquan
Derrick
Devon
Dacorius
Darius
Desean
Denzel
Dekota
Dorrel
Devin
Derry
Darrius
DeQuan
DeLorian
Devonte
Denzel
Damon
Dennis 
Darion
Devan
DeAngelo
Daniel
Devohn
David
Delroy
Darren
Demetrius
Danzel
DJ
Dorian
Dimonic
Donavin
Desho

Deon
Derek
Damon
Derrick
Dylan
Dairyon
Darius
DB
Dekwan
Diamonte
Denzel
Drew
Davante
Darius
Derek
Darnell
Dennis
Dwayne
Darius
DeOndre
Darius
Donald
Dalvin
Dionte
Dallas
Darabie
DeSean
Deven
Drew
Dillon
Deonta
Daniel
Deion
Dylan
Deion
Deandre
Damond
Demond
David
Dalonte
Davis
DeAndre
Devonte
Dajon
Doug
Doug
Dakota
Dexter
Dylan
Deine
Danikeei
Danarius
Donovan
Diondre
David
Darnell
Dimonic
Demeco
Duncan
Demarcus
Darrius 
Derek
Donavan
DeAnthony
Daniel
Dalton
Donnie
Dimitri
Davon
Dante
Daryl
Dvonte
Daniel
Devon
Dylan
Darius
Darius
Dennis
Devlin
Demetrius
Damien
Desmond
Dougladson
Dawaune
Denard
Danny
Dominick
Dreshown
Dakeavis
DeJean
Delshawn
Dashawn
Darius
Dante
Deonshay
Duval
Dimitri
Davon
Deion
Dillon
Donnie
D'Angelo
Del
Douglas
Damu
Demetri
Darion
Dalton
Drewe
David
DeShaun
Darzil
Da'quan
Daryl
Deion
Dennis
Daiqwon
D'Andre
Derick
Dra
Devin
Derek
Darius
Devyn
Devon
Daquan
Dylan
Devin
Donovan
David
Darryl
Darius
Donovan
Dre
Devon
Deyanco
DeShon
Drew 
Darrell
Devin
David
Dominique
Davion

Demetrius
Dakota
Dylan
Demetrius
Delvin
Darryl
Desmond
Dylan
Darryl
Demiere
Drew
Dante
Demetrius
Deon
Daniel
Dante
Dustin
Dylan
Damon
Dentrell
Doniel
Douglas
Dwayne
Darren
Darryl
Delano
Delmon
Dwight
Demetrius
Diavalo
D.J.
Darrin
Devonta
Darius
Demarkus
Deric
Darrel
Daniel
Derrick
Dabness
Devin
Devante
Deandre
Derek
Dustian
Derek
D'Leon
Donnell
Darrell
DeAndre
Derrick
Damien
Dennis
Drew
DeVonte
David
Dylan
Dylan
Derek
Dimitri
Dennis
Darzil
Darrell
DeVontae
Dillon
Devontay
Dequindre
Dwaye
Daryl
David
Dion
Dirk
Daron
Davante
Daniel
Dallas
Dale
DeAndre
Dustin
Dorian
Denzell
David
D'Vante
Dominique
Dajour
Diontre
Devin
Diaheem
Doug
Desman
Darius
David
Dawan
Dominic
Devin
Derek
Devonte
DJ
Darrin
Darius
Dontrill
Devonte
Delryn
Dillon
Desmond
Dernard
Donald
Denzell
Daniel
Devin
David
Deshon
Dominique
Destry
Davante
Delrece
Derek
Dillon
Derek
Dwayne
Dallas
Dennis
Darian
Deon
Drew
Deandre
Demazio
Dorsey
Dinsdale
Dominique
D.J.
Duke
Daniel
Darien
David
Dinero
David
DJ
Davis
Dylan
David
David
Dam

Darrion
Devante
David
Dalton
Dietrich
Daniel
De'Jhion
Daniel
Daryl
Dermel
Derrick
Devon
Dylan
Drake
Donald
Diante
Denver
Davarius
Donnovan
Devin
Dylan
Desmond 
Drake
D.J.
Dana
Dalton
Dante
Dwayne
Devon
Deion
Dante
Drayke
David
Dalton
Davien
David
Devin
Davendrick
Dawson
Dylan
Dante
Devon
Drew
Dennis
Donzae
Devontay
Darius
Donta
Donnie
Doug
Daniel
Desmond
Donnell
David
Darius
Damario
Derrick
Deandre
Danny
Demetrius
Dadrian
Demonquez
Daniel
De'Jai
Des Jahvonni
Darius
Daekwon
Darius
Daviyon
Dhani
Desmond
Darwin
Donald
D.J.
Dorien
Dunn
Dylon
Demetrius
Devin
Deandre
Dekwon
Deano
DeAndre
Devaughn
De'Ontay
Dan
Daniel
David
David
DeAndre
David
DaVone
Daniel
Dimitrius
David
Dajour
Damon
Daulton
Djimon
Dorrius
Dru
David
Dionte
Devin
Deverion
Daniel
Dashawn
Drew
DeMichael
Devin
Devin
Daniel
Desherrius
Drew
Dunya
Davel
Deion
Deland
Dashon
Daren
Dwight
D.J.
Datryan
Darran
Deanthony
Darrin
Darren
Daniel
Deionne
Donte
Davion
Deondre
Darian
Durron
Dayon
Demar
Dayshawn
Danny
Devin
Darius
David
Danny
De

Deandre
Dezman
Dorian
Dylan
Deon
David
Daron
Dayton
Dawan
David
Darius
Dane
Donovan
Demarcus
Devin
Dante
DJ
Darin
Desmond
Demaurez
Darius
Darrell
Dante
Donnell
Dekendrick
Dariyan
Daijuane
Darian
Dimitri
Dalton
Damico
Derek
Dakota
Diego
Daniel
Deuce
Drexlan
Dylan
Dylan
Drew
Dion
Demarcus
Darius
Damean
Daniel
Delon
Deon
Devin
Darius
Davion
Demetrius
David
DeQuece
Darius
DaJour
Dante
Dusty
Derik
DJ
Dean
Davis
Daniel
Dylan
Dominique
Dalton
Darrion
Dwain
Davone
Devin
Devin
Dewuan
Davis
Devon
Dagen
Dorian
Declan
Donovan
Dashaun
Demarr
Dawson
D'Andre
Donaven
Dino
D.J.
Daniel
Donovan
Deshawn
Dylan
Dylan
DeAngelo
Drew
Davion
Diego
Dylan
Dalton
Donavyn
D.J.
Derrick
Dominic
D.J.
Dillon
Deonta
Dylan
Dominic
Decarlo
David
DeAndre
Deiontrae
Damon
Demetrius
Damonte
De'Angelo
Dalton
Dillon
Dylan
Drew
Dayton
Damarco
Dawson
DJ
Dante
Dylan
Dillon
Da'Marcus
D'shon
Dujuan
Deoveon
David
Dyquan
Deandre
Demoundre
Dennis
Davon
Daryn
Dylan
David
Damian
Dorian
Deondre
Darius
Daniel
Degerrick
Daryl
Dylan
Deante
D

In [11]:
# Example sorting
# Error because Sort exceeded memory limit
sort_result = players247.objects.order_by('+Player.FirstName','-Player.LastName')
for each in sort_result:
    try:
        print(each.Player.FirstName, each.Player.LastName)
    except OperationFailure:
        continue

OperationFailure: Executor error during find command :: caused by :: Sort exceeded memory limit of 104857600 bytes, but did not opt in to external sorting., full error: {'ok': 0.0, 'errmsg': 'Executor error during find command :: caused by :: Sort exceeded memory limit of 104857600 bytes, but did not opt in to external sorting.', 'code': 292, 'codeName': 'QueryExceededMemoryLimitNoDiskUseAllowed'}

#### Store certain data in pandas

In [12]:
# from playerModel import *
from collections import defaultdict
import pandas as pd
from collections import Counter

In [13]:
df1 = pd.DataFrame({'FullName':full_list})
df1.head()

,FullName
0,Leonard Fournette
1,Myles Garrett
2,Jabrill Peppers
3,Cam Robinson
4,Da'Shawn Hand


In [14]:
df2 = pd.DataFrame({'Height':height_list})
df2.head()

,Height


## Statistic works

#### Collect the data we need

In [15]:
# 2 mins
records = players247.objects()
states = set()
positions =  set()
player_list = []
for record in records:
    states.add(record.Player.Hometown.State)
    positions.add(record.Player.PrimaryPlayerPosition.Abbreviation)
    player_list.append((record.Player.Hometown.State,record.Player.PrimaryPlayerPosition.Abbreviation))

#### Count the number of players from each state

In [16]:
for state in states:
    # Example filter (can be converted to a function)
    result = players247.objects.filter(Player__Hometown__State=state)
    print(state,":",result.count())

Alabama : 1876
Delaware : 105
Manitoba : 2
Maryland : 1016
Hawaii : 314
Puerto Rico : 1
Denmark : 2
Australia : 40
Kansas : 759
Nebraska : 324
Quebec : 26
West Virginia : 174
North Rhine-Westphalia : 1
New Mexico : 130
Ontario : 63
Sweden : 6
Nova Scotia : 1
New South Wales : 8
Finland : 2
England : 7
Illinois : 1469
North Dakota : 51
Montana : 40
Germany : 23
Arizona : 1039
Victoria : 9
Utah : 538
New York : 847
Idaho : 193
Connecticut : 295
South Africa : 1
Guam : 1
Alberta : 3
Alaska : 31
North Carolina : 1725
Wyoming : 34
Missouri : 735
California : 6898
Colorado : 437
Arkansas : 679
United Kingdom : 1
Kentucky : 551
Minnesota : 596
South Dakota : 31
British Columbia : 35
Pennsylvania : 2227
Oklahoma : 668
Baden-WÃ¼rttemberg : 2
Indiana : 1068
Louisiana : 1934
New Zealand : 2
New Hampshire : 32
Na : 16
Canada : 23
Rhode Island : 20
China, Peoples Republic of : 1
Ohio : 2271
Massachusetts : 373
Georgia : 4141
Wisconsin : 517
None : 24
American Samoa : 26
Mississippi : 1666
Vermont :

#### Count the number of players from different states for a particular position 

In [17]:
position_states = defaultdict(list)

for position, state in player_list:
        position_states[state].append(position) 

for key,values in position_states.items():
    print(key,":",list(Counter(values)),"\n")
 #   print(key,":",Counter(values),"\n")
#     print(key,":",values,"\n")

RB : ['Louisiana', 'Florida', 'Alabama', 'California', 'Georgia', 'North Carolina', 'New York', 'Colorado', 'Illinois', 'Texas', 'Oklahoma', 'Pennsylvania', 'Mississippi', 'Kansas', 'Virginia', 'New Jersey', 'Ohio', 'Missouri', 'Kentucky', 'Maryland', 'Massachusetts', 'South Carolina', 'Utah', 'Tennessee', 'British Columbia', 'Minnesota', 'Arkansas', 'Connecticut', 'Oregon', 'Indiana', 'Nebraska', 'Arizona', 'Hawaii', 'Nevada', 'Michigan', 'Wisconsin', 'Iowa', 'New Hampshire', 'Na', 'West Virginia', 'District Of Columbia', 'New Mexico', 'Washington', 'Wyoming', 'Idaho', 'Delaware', 'New South Wales', 'Montana', 'Maine', 'North Dakota', 'Rhode Island', 'Ontario', 'South Dakota', 'China, Peoples Republic of', 'England', 'Guam', 'Puerto Rico', 'Alaska', 'Canada', 'Nova Scotia', 'Germany'] 

WDE : ['Texas', 'Georgia', 'Michigan', 'Florida', 'Maryland', 'Arizona', 'North Carolina', 'Virginia', 'Mississippi', 'Pennsylvania', 'Oklahoma', 'Ohio', 'Illinois', 'Oregon', 'Nevada', 'South Carolina